In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('../')

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from transformers import DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
import json
import os
import random
from sklearn.model_selection import train_test_split
import torch
from tqdm.notebook import tqdm

In [3]:
from model import V2JRewardModelConfig, V2JRewardModel
AutoConfig.register('V2JRewardModel', V2JRewardModelConfig)
AutoModel.register(V2JRewardModelConfig, V2JRewardModel)

In [4]:
DATA_DIR = "."
SEED = 42
MAX_SEQ_LEN = V2JRewardModel.MAX_SEQ_LEN

In [5]:
model_name = "models/v2j-reward-large"

In [45]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [46]:
dataset_files = [
    "reward_dataset_v2j-vectors-to-jokes.json"
]

final_dataset = []
for dataset_file in dataset_files:
    with open(os.path.join(DATA_DIR, dataset_file), 'r') as f:
        final_dataset.extend(json.load(f))

In [57]:
class RedditRewardDataset(Dataset):
    def __init__(self, tokenizer, dataset, dataset_type):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.dataset_type = dataset_type

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        return tokenizer("Human: " + sample["query"], "Assistant: " + sample[self.dataset_type], truncation=True, max_length=MAX_SEQ_LEN)

    def __len__(self):
        return len(self.dataset)

In [58]:
train_final_dataset, test_final_dataset = \
        train_test_split(final_dataset, test_size=0.2, random_state=SEED)

In [59]:
train_dataset_chosen = RedditRewardDataset(tokenizer, train_final_dataset, "chosen")
train_dataset_rejected = RedditRewardDataset(tokenizer, train_final_dataset, "rejected")

test_dataset_chosen = RedditRewardDataset(tokenizer, test_final_dataset, "chosen")
test_dataset_rejected = RedditRewardDataset(tokenizer, test_final_dataset, "rejected")

In [60]:
# Do not shuffle as we are iterating chosen and rejected together 
# Shuffle was performed for train/test split
batch_size = 4

train_dataloader_chosen = DataLoader(train_dataset_chosen, batch_size=batch_size, shuffle=False, 
                                     collate_fn=DataCollatorWithPadding(tokenizer))
train_dataloader_rejected = DataLoader(train_dataset_rejected, batch_size=batch_size, shuffle=False, 
                              collate_fn=DataCollatorWithPadding(tokenizer))

test_dataloader_chosen = DataLoader(test_dataset_chosen, batch_size=batch_size, shuffle=False, 
                              collate_fn=DataCollatorWithPadding(tokenizer))
test_dataloader_rejected = DataLoader(test_dataset_rejected, batch_size=batch_size, shuffle=False, 
                              collate_fn=DataCollatorWithPadding(tokenizer))

In [61]:
device = torch.device("cuda")

In [70]:
epochs = 1
lr = 2e-6

#model = AutoModelForSequenceClassification.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, config=config)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

#loss_fn = torch.nn.MarginRankingLoss()
loss_fn = lambda chosen, rejected: -torch.nn.functional.logsigmoid(chosen - rejected)

In [71]:
def evaluate(model, eval_dataloaders, loss_fn):
    eval_dataloader_chosen, eval_dataloader_rejected = eval_dataloaders

    model.eval()
    
    correct = 0
    total = 0
    eval_loss = 0
    
    with torch.no_grad():
        for i, (batch_chosen, batch_rejected) in \
                enumerate(tqdm(
                    zip(test_dataloader_chosen, test_dataloader_rejected), 
                    "Evaluation", total=len(test_dataloader_chosen))):
            batch_chosen = batch_chosen.to(device)
            batch_rejected = batch_rejected.to(device)
            
            chosen_outputs = model(batch_chosen)
            rejected_outputs = model(batch_rejected)
            
            loss = loss_fn(chosen_outputs.logits, rejected_outputs.logits)
            eval_loss += loss.sum().item()
            correct += (chosen_outputs.logits > rejected_outputs.logits).sum().item()
            total += loss.shape[0]
    
    print(f"Eval loss: {eval_loss/total:.2f}")
    print(f"Eval accuracy: {correct/total:.2f}")

In [72]:
evaluate(model, (test_dataloader_chosen, test_dataloader_rejected), loss_fn)

Evaluation:   0%|          | 0/89 [00:00<?, ?it/s]

Eval loss: 0.65
Eval accuracy: 0.65


In [73]:
def train(model, train_dataloaders, eval_dataloaders, epochs, loss_fn, optimizer):
    train_dataloader_chosen, train_dataloader_rejected = train_dataloaders
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}:")
        model.train()
        
        train_loss = 0.0
        total = 0
        
        for i, (batch_chosen, batch_rejected) in \
                enumerate(tqdm(
                    zip(train_dataloader_chosen, train_dataloader_rejected), 
                    "Training", total=len(train_dataloader_chosen))):

            batch_chosen = batch_chosen.to(device)
            batch_rejected = batch_rejected.to(device)

            chosen_outputs = model(batch_chosen)
            rejected_outputs = model(batch_rejected)
            loss = loss_fn(chosen_outputs.logits, rejected_outputs.logits)
            
            train_loss += loss.sum().item()
            total += loss.shape[0]

            optimizer.zero_grad()
            loss.sum().backward()
            optimizer.step()
            
        print(f"Train loss: {train_loss/total:.2f}")
        evaluate(model, eval_dataloaders, loss_fn)

In [74]:
train(model, (train_dataloader_chosen, train_dataloader_rejected),
             (test_dataloader_chosen, test_dataloader_rejected),
      epochs, loss_fn, optimizer)

Epoch 1:


Training:   0%|          | 0/354 [00:00<?, ?it/s]

Train loss: 0.56


Evaluation:   0%|          | 0/89 [00:00<?, ?it/s]

Eval loss: 0.48
Eval accuracy: 0.76


In [75]:
def save_hf_model(model, tokenizer, model_path):
    model.save_pretrained(model_path)
    model.config.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

In [76]:
save_hf_model(model, tokenizer, "models/v2j-reward-large")